In [1]:
from termcolor import colored
import math
import nltk
import math
import re
import string
import numpy as np
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from tqdm import tqdm

wremove = ['-','â€','â€¦','”']
filedata = []
def remove_prepositions(text):
    tokens = nltk.word_tokenize(text)
    tagged = nltk.pos_tag(tokens)
    filtered = [word for word, pos in tagged if pos not in ['IN', 'TO']]
    return ' '.join(filtered)
# CREATING INSTANCES OF STEMMING AND LEMMITIZATION
stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()
stopwords = ["was","wasn't","was not","donot","don't","there","here","never","n't","s","been","will","would","wouldn't","hasn't","has not","haven't","have not","doesn't","had","hadn't","had not","upto","and","or","not","nt","that","this","it","i","why","who","how","what","when","he","she","their","you","but","because","then","which","him","her","their","where","could","an"]
translator = str.maketrans('', '', string.punctuation.replace('-', ''))
# READING STOP WORDS FROM FILE
with open("C:\\Users\\DELL\\Desktop\\VectorSpace\\Stopword-List.txt", "r") as f:
    contents = f.read()
    words = contents.split()

    # Store the words in an array
    for word in words:
        stopwords.append(word)

documents = []
wholecontent = ""
# READING ALL 30 FILES AND APPLY PREPROCESSING 
for x in range(1, 31):
    t = "C:\\Users\\DELL\\Desktop\\VectorSpace\\{}.txt".format(x)
    with open(t, "r") as f:
        contents = f.read()
        # FOR REMOVING LINKS AND SPECIAL CHARACTRS FROM FILE
        contents = re.sub(r"&nbsp;.*\n", " ", contents)
        contents = re.sub(r"â€¢.*\n", " ", contents)
        # STEMMING PROCESS
        contents = nltk.word_tokenize(contents)
        contents = [stemmer.stem(word) for word in contents]
        contents = ' '.join(contents)
        # LEMMITIZATION PROCESS
        contents = " ".join([lemmatizer.lemmatize(word, pos='v') if pos.startswith('V') else lemmatizer.lemmatize(word) for word, pos in nltk.pos_tag(nltk.word_tokenize(contents))])
        # REMOVING PUNCTUATION MARKS
        contents = contents.translate(translator)
        # APPLYING CASE FOLDING
        contents = contents.lower()
        # REMOVING PREPOSITIONS
        contents = remove_prepositions(contents)
        contents = ' '.join(word for word in contents.split() if word not in stopwords)
        # GATHERING ALL PREPROCESSED DATA
        filedata.append(contents)
        wholecontent += contents
        contents = ""
# BUILDING INDEXES
wholecontent = wholecontent.split()
wholecontent = set(wholecontent)
wholecontent = sorted(list(wholecontent))
wholecontent = [element for element in wholecontent if element not in wremove]

term_matrix = [[0 for j in range(64)] for i in range(len(wholecontent)+1)]

term_matrix[0][0] = 'Terms'

for x in range(1, 31):
        col = 'D' + str(x)
        term_matrix[0][x] = col
term_matrix[0][31] = 'df'
term_matrix[0][32] = 'D/df'
term_matrix[0][33] = 'IDF'

i = 1
for x in range(34, 64):
        col = 'D' + str(i)
        i = i+1
        term_matrix[0][x] = col
for x in range(0, len(wholecontent)):
    term_matrix[x+1][0]=wholecontent[x]
    
for x in range(0, len(wholecontent)):
    for y in range(0, 30):
        if wholecontent[x] in filedata[y]:
            term_matrix[x+1][y+1] = filedata[y].count(wholecontent[x])
sum = 0

for x in range(0, len(wholecontent)):
    for y in range(0, 30):
        if term_matrix[x + 1][y + 1] !=0:
            sum = sum + 1
    term_matrix[x+1][31] = sum
    sum = 0

term_matrix = [row for row in term_matrix if row[31] != 0]

for x in range(0, len(term_matrix)-1):
    term_matrix[x+1][32] = 30 / term_matrix[x+1][31]
    
for x in range(0, len(term_matrix)-1):
    term_matrix[x+1][33] = math.log(term_matrix[x+1][32], 10)
    
for x in range(0, len(term_matrix)-1):
    z = 0
    for y in range(34, len(term_matrix[0])):
        z = z + 1
        term_matrix[x+1][y] = term_matrix[x+1][z] * term_matrix[x+1][33]
doclen = [0] * 30

for col in range(34, len(term_matrix[0])):
    sum = 0
    for row in range(1, len(term_matrix)):
        sum = sum + pow(term_matrix[row][col], 2)
    doclen[col-34] = math.sqrt(sum)
        

for x in range(0, len(term_matrix)):
    for y in range(0, 64):
        print(term_matrix[x][y], " ", end='')
    print('\n')



Terms  D1  D2  D3  D4  D5  D6  D7  D8  D9  D10  D11  D12  D13  D14  D15  D16  D17  D18  D19  D20  D21  D22  D23  D24  D25  D26  D27  D28  D29  D30  df  D/df  IDF  D1  D2  D3  D4  D5  D6  D7  D8  D9  D10  D11  D12  D13  D14  D15  D16  D17  D18  D19  D20  D21  D22  D23  D24  D25  D26  D27  D28  D29  D30  

0  18  9  14  3  7  8  8  9  7  6  6  8  18  27  5  2  4  9  2  3  0  11  8  5  9  12  15  14  4  8  29  1.0344827586206897  0.014723256820706378  0.2650186227727148  0.1325093113863574  0.2061255954898893  0.04416977046211913  0.10306279774494465  0.11778605456565103  0.11778605456565103  0.1325093113863574  0.10306279774494465  0.08833954092423826  0.08833954092423826  0.11778605456565103  0.2650186227727148  0.3975279341590722  0.0736162841035319  0.029446513641412757  0.058893027282825514  0.1325093113863574  0.029446513641412757  0.04416977046211913  0.0  0.16195582502777017  0.11778605456565103  0.0736162841035319  0.1325093113863574  0.17667908184847653  0.22084885231059567  0.2

In [2]:
import re
import ipywidgets as widgets
from IPython.display import display

# Define the input widgets.
query_input = widgets.Text(placeholder='Enter a query')

# Define the output widgets.
output_label = widgets.Label()
output_list = widgets.SelectMultiple()

# Define the button widget and its click event handler.
button = widgets.Button(description='Search')

def on_button_click(_):
    query = query_input.value
    if not query:
        output_label.value = 'Error: Query is not valid.'
        
    else:
        cossim = [0] * 30
        qlen = 0
        qvector = [0] * len(term_matrix)
        qvector[0]= 'Term'
        parse = query.split()
        for x in range(0, len(parse)):
            #APPLYING STEMMING AND LEMMITIZATION TO QUERY TERMS 
            parse[x] = parse[x].lower()
            parse[x] = stemmer.stem(parse[x])
            parse[x] = lemmatizer.lemmatize(parse[x])
        for x in range(1, len(term_matrix)):
            if term_matrix[x][0] in parse:
                qvector[x] = parse.count(term_matrix[x][0])*term_matrix[x][33]
                qlen = qlen + pow(qvector[x], 2)
        qlen = math.sqrt(qlen)
        for x in range(0, 30):
            sum = 0
            for y in range(1, len(qvector)):
                sum = sum + qvector[y]*term_matrix[y][x+34]
            cossim[x] =  sum/doclen[x]*qlen
        numpy_cossim = np.array(cossim)
        cut_off_score = np.mean(numpy_cossim) + (0.05 * np.std(numpy_cossim))
        result_set = []
        for x in range(0, 30):
            if numpy_cossim[x] >= cut_off_score:
                result_set.append(x+1)
        output_label.value = 'Results:'
        output_list.options = result_set

button.on_click(on_button_click)

# Create a container to hold the widgets.
input_box = widgets.VBox([query_input, button])
output_box = widgets.VBox([output_label, output_list])
container = widgets.HBox([input_box, output_box])

# Display the container.
display(container)
